<a href="https://colab.research.google.com/github/AKASH4671/Sentiment-Analysis-on-Financial-News-and-Its-Impact-on-Stock-Prices/blob/main/01_yahoo_apple_news_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Libraries**

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse
from datetime import datetime, timedelta

**Setup**

In [ ]:
# Set up ChromeDriver path (for Jupyter Notebook)
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Optional: if don't want to open browser window
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

In [ ]:
#  Define path to chromedriver.exe (**note** - same folder)
service = Service("chromedriver.exe")
# Launch the Chrome driver with options
driver = webdriver.Chrome(service=service, options=chrome_options)

In [ ]:
# Yahoo Finance AAPL News
driver.get("https://finance.yahoo.com/quote/AAPL/news?p=AAPL")
time.sleep(3)
print(driver.title)
# driver.quit() # Always close the driver at the end

Apple Inc. (AAPL) Latest Stock News & Headlines - Yahoo Finance


In [ ]:
# Scroll and load more news
scroll_pause_time = 3
scroll_count = 150  # can increase for more headlines

last_height = driver.execute_script("return document.body.scrollHeight")
scroll_pause_time = 1.5
retries = 0

for i in range(100):  # Try to scroll 100 times max
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)

        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            retries += 1
            if retries > 3:  # Stop after 3 tries if nothing new is loading
                print(f" No new content after {i} scrolls. Ending scroll.")
                break
        else:
            retries = 0
            last_height = new_height

    except TimeoutException:
        print(" Timeout occurred during scroll. Exiting.")
        break

print(" Done scrolling.")

# Scrape news headlines
print(" Searching for updated news containers...")

articles = driver.find_elements(By.XPATH, '//div[@class="content yf-zt3p0l"]/a')
print(f" Found {len(articles)} headlines")

news_data = []

from dateutil.relativedelta import relativedelta
from dateutil.parser import parse
from datetime import datetime, timedelta

# scraping block
news_data = []

for article in articles:
    try:
        headline = article.find_element(By.TAG_NAME, "h3").text.strip()
        link = article.get_attribute("href")

        #  Now get the parent div and find date text
        parent_div = article.find_element(By.XPATH, "..")  # go to div.content
        date_element = parent_div.find_element(By.CLASS_NAME, "publishing")
        date_text = date_element.text.strip()  # Example: 'Yahoo Finance • 2 hours ago'

        #  Extract relative time from the date string
        if "•" in date_text:
            relative_time = date_text.split("•")[-1].strip()
        else:
            relative_time = date_text.strip()

        # Convert "2 hours ago", "3 days ago", "Yesterday" → datetime
        now = datetime.now()
        if "yesterday" in relative_time.lower():
            pub_date = now - timedelta(days=1)
        elif "hour" in relative_time:
            num = int(relative_time.split()[0])
            pub_date = now - timedelta(hours=num)
        elif "minute" in relative_time:
            num = int(relative_time.split()[0])
            pub_date = now - timedelta(minutes=num)
        elif "day" in relative_time:
            num = int(relative_time.split()[0])
            pub_date = now - timedelta(days=num)
        else:
            pub_date = ""  # fallback if unknown format

        news_data.append({
            "date": pub_date.strftime("%Y-%m-%d") if pub_date else "",
            "headline": headline,
            "url": link,
            "ticker": "AAPL",
            "source": "Yahoo Finance"
        })

    except Exception as e:
        print(" Skipping due to error:", e)
        continue

driver.quit()

# Convert to DataFrame and save
df = pd.DataFrame(news_data)
df.to_csv("apple_news.csv", index=False)
print(f" Scraped {len(df)} Apple news articles and saved to 'apple_news.csv'")